In [1]:
import os
import getpass

os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")

In [2]:
os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ_API_KEY:")

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore


In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="en",
    num_workers=1,
    premium_mode = True
)

In [5]:
documents = parser.load_data([r"C:\Users\nisha\Desktop\RAG_Assign\RAG_Assign\iesc111.pdf"])

Parsing files: 100%|██████████| 1/1 [00:20<00:00, 20.28s/it]


In [52]:
len(documents)

14

In [ ]:
print(documents[2].text[::])

In [ ]:
print(documents[3].text[::])

In [35]:
from llama_index.llms.groq import Groq

In [36]:
llm = Groq(model="llama3-70b-8192", api_key="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")
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


In [ ]:
from llama_index.core import Settings


In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
print(resp)

In [46]:
from llama_index.core.node_parser import MarkdownElementNodeParser


In [ ]:
"""
splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

nodes"""

In [ ]:
"""
splitter1 = MarkdownNodeParser(
    include_metadata= False,
    include_prev_next_rel= True,
)

nodes_markdown = splitter1.get_nodes_from_documents(documents)

nodes_markdown[1]
"""

In [82]:
node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8)

In [83]:
nodes_markdown_element = node_parser.get_nodes_from_documents(documents=documents)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [86]:
nodes_markdown_element[7]

IndexNode(id_='db060258-4878-4705-be5c-386b4978e71f', embedding=None, metadata={'col_schema': 'Column: State\nType: string\nSummary: None\n\nColumn: Substance\nType: string\nSummary: None\n\nColumn: Speed in m/s\nType: number\nSummary: None'}, excluded_embed_metadata_keys=['col_schema'], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7a502ce8-cada-4f01-b621-6b8f4ae55c54', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='9d0d77c9411a751597c9f7f9be255d1ae84b92311c71dcf545bdc170e8689b54'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='56f63ebd-6025-4439-8286-993512753375', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d8f6433e378187647f66887e09562443b2f6efd17af31115e71054d502320695'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8ea8291c-cc0c-4de4-a8a9-be49d3a6b694', node_type=<ObjectType.TEXT: '1'>, metadata={'table_df': "{' State   ': {0: ' Solids  ', 1: '         ', 2: '         ', 3: '       

In [87]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes_markdown_element)

In [88]:
index_markdown_element = VectorStoreIndex(nodes=base_nodes+objects)

In [ ]:
%pip install llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

In [61]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker


In [89]:
reranker = FlagEmbeddingReranker(
    top_n=3,
    model="BAAI/bge-reranker-large",
)

In [156]:
markdown_element_query_engine = index_markdown_element.as_query_engine(
    similarity_top_k=5,
    verbose=True
)

In [157]:
eq = "What is the speed of sound in aluminium?"

response_element1 = markdown_element_query_engine.query(eq)
print(response_element1)

Retrieval entering 8ea8291c-cc0c-4de4-a8a9-be49d3a6b694: TextNode
Retrieving from object TextNode with query What is the speed of sound in aluminium?
The speed of sound in aluminium is 6420 m/s.


In [ ]:
eq2 = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."
response_element2 = markdown_element_query_engine.query(eq2)
print(response_element2)

In [131]:
from llama_index.core.query_engine import RetrieverQueryEngine


In [132]:
from typing import List, Dict, Any
from llama_index.core import VectorStoreIndex
from llama_index.core.base.response.schema import Response
from llama_index.core.schema import NodeWithScore
from llama_index.llms.groq import Groq
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import FunctionTool

In [162]:
import logging 

In [176]:
class WorkflowState:
    def __init__(self, query: str, initial_response: str):
        self.query = query
        self.initial_response = initial_response
        self.additional_info = ""
        self.final_answer = ""
        self.needs_info = False


def check_completeness(state: WorkflowState, llm: Groq) -> WorkflowState:
    prompt = f"""
    Given the following query and initial response, determine if the response contains 
    all necessary information to fully answer the query. If not, specify what additional 
    information is needed.

    Query: {state.query}
    Initial Response: {state.initial_response}

    Output your response in the following format:
    Complete: [Yes/No]
    Additional Info Needed: [If No, specify what additional information is needed]
    """

    response = llm.complete(prompt)

    # Extract 'Complete' and 'Additional Info Needed' fields from response
    lines = response.split('\n')
    for line in lines:
        if 'Complete' in line:
            state.needs_info = 'No' in line
        elif 'Additional Info Needed' in line and state.needs_info:
            state.additional_info = line.split(': ')[1].strip()

    return state


def retrieve_additional_info(state: WorkflowState, query_engine: RetrieverQueryEngine) -> WorkflowState:
    if state.needs_info:
        additional_info_query = state.additional_info
        response = query_engine.query(additional_info_query)

        if isinstance(response, str):
            state.additional_info = response.strip()
        else:
            raise TypeError("Expected string response from query engine.")
    
    return state


def generate_final_answer(state: WorkflowState, llm: Groq) -> WorkflowState:
    prompt = f"""
    Given the following query, initial response, and additional information (if any),
    generate a complete and accurate answer.

    Query: {state.query}
    Initial Response: {state.initial_response}
    Additional Information: {state.additional_info}

    Complete Answer:
    """

    response = llm.complete(prompt)

    if isinstance(response, str):
        state.final_answer = response.strip()
    else:
        raise TypeError("Expected string response from generate_final_answer.")

    return state


def create_workflow(query_engine: RetrieverQueryEngine, llm: Groq):
    tools = [
        FunctionTool.from_defaults(
            fn=lambda x: check_completeness(x, llm),
            name="check_completeness",
            description="Check if the initial response is complete"
        ),
        FunctionTool.from_defaults(
            fn=lambda x: retrieve_additional_info(x, query_engine),
            name="retrieve_additional_info",
            description="Retrieve additional information if needed"
        ),
        FunctionTool.from_defaults(
            fn=lambda x: generate_final_answer(x, llm),
            name="generate_final_answer",
            description="Generate the final answer"
        )
    ]

    agent = OpenAIAgent.from_tools(
        tools,
        llm=llm,
        verbose=True
    )

    return agent


def solve_query_with_workflow(query: str, query_engine: RetrieverQueryEngine, llm: Groq) -> str:
    # Get initial response
    initial_response = query_engine.query(query)

    # Create initial state
    state = WorkflowState(query=query, initial_response=str(initial_response))

    # Create and run the workflow
    agent = create_workflow(query_engine, llm)

    # Define the task for the agent
    task = f"""
    Given the following query and initial response, use the available tools to ensure a complete and accurate answer is provided.
    If the initial response is incomplete, retrieve additional information and generate a final answer.

    Query: {state.query}
    Initial Response: {state.initial_response}

    Use the tools in this order:
    1. check_completeness
    2. retrieve_additional_info (if needed)
    3. generate_final_answer

    Return the final answer.
    """

    # Run the agent
    response = agent.chat(task)

    # Process the response to extract the final answer
    if hasattr(response, 'response'):
        final_answer = response.response.strip()
    else:
        raise ValueError("Unexpected response format from agent.")

    return final_answer

In [177]:
query = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."
#logger.info(f"Processing query: {query}")
final_answer = solve_query_with_workflow(query, markdown_element_query_engine, llm)
#logger.info(f"Final answer: {final_answer}")
print(final_answer)

Batches: 100%|██████████| 1/1 [00:03<00:00,  3.16s/it]
2024-09-29 06:21:56,664 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Added user message to memory: 
    Given the following query and initial response, use the available tools to ensure a complete and accurate answer is provided.
    If the initial response is incomplete, retrieve additional information and generate a final answer.

    Query: Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child.
    Initial Response: The speed of sound in air is given as 344 m s⁻¹. Let's assume the length of the aluminium rod is 'L'. The time taken by the sound wave to reach the second child in air is t₁ = L / 344. 

The speed of sound in a medium depends on the nature of the medium. Since the speed of sound in aluminium is not given, let's assume it to be 'v'. The time taken by the sound wave to reach the second child in aluminium is t₂ = L / v. 

Now, we need to find the ratio of t₁ and t₂. 

t₁ / t₂ = (L / 344) / (L / v) 
t

2024-09-29 06:21:58,042 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: check_completeness with args: {"x":"The speed of sound in air is given as 344 m s⁻¹. Let's assume the length of the aluminium rod is 'L'. The time taken by the sound wave to reach the second child in air is t₁ = L / 344. The speed of sound in a medium depends on the nature of the medium. Since the speed of sound in aluminium is not given, let's assume it to be 'v'. The time taken by the sound wave to reach the second child in aluminium is t₂ = L / v. Now, we need to find the ratio of t₁ and t₂. t₁ / t₂ = (L / 344) / (L / v) t₁ / t₂ = v / 344. So, the ratio of times taken by the sound wave in air and in aluminium to reach the second child is v / 344."}
Got output: Error: 'str' object has no attribute 'query'



2024-09-29 06:21:59,220 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


...

=== Calling Function ===
Calling function: generate_final_answer with args: {"x":"The speed of sound in air is given as 344 m s⁻¹. Let's assume the length of the aluminium rod is 'L'. The time taken by the sound wave to reach the second child in air is t₁ = L / 344. The speed of sound in a medium depends on the nature of the medium. Since the speed of sound in aluminium is not given, let's assume it to be 'v'. The time taken by the sound wave to reach the second child in aluminium is t₂ = L / v. Now, we need to find the ratio of t₁ and t₂. t₁ / t₂ = (L / 344) / (L / v) t₁ / t₂ = v / 344. So, the ratio of times taken by the sound wave in air and in aluminium to reach the second child is v / 344. The speed of sound in aluminium is approximately 6420 m/s."}
Got output: Error: 'str' object has no attribute 'query'



2024-09-29 06:22:40,953 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-09-29 06:22:40,953 - INFO - Retrying request to /chat/completions in 22.000000 seconds
2024-09-29 06:23:03,614 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: generate_final_answer with args: {"x":"The ratio of times taken by the sound wave in air and in aluminium to reach the second child is 6420 / 344."}
Got output: Error: 'str' object has no attribute 'query'



2024-09-29 06:23:03,996 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-09-29 06:23:03,997 - INFO - Retrying request to /chat/completions in 24.000000 seconds
2024-09-29 06:23:28,784 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


The final answer is: The ratio of times taken by the sound wave in air and in aluminium to reach the second child is 18.6.


In [161]:
eq2 = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."
final_answer = solve_query_with_workflow(eq2, markdown_element_query_engine, llm)

print(final_answer)

Here is the complete and accurate answer:

Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child.

Let's assume the length of the aluminium rod is L. The speed of sound in air is 344 m/s. The speed of sound in aluminium is approximately 6420 m/s.

Time taken by the sound wave in air = L / 344 m/s
Time taken by the sound wave in aluminium = L / 6420 m/s

Now, we can find the ratio of the times taken:

Ratio = (Time taken by the sound wave in air) / (Time taken by the sound wave in aluminium)
= (L / 344 m/s) / (L / 6420 m/s)
= (L / 344) × (6420 / L)
= 6420 / 344
= 18.65

So, the ratio of the times taken by the sound wave in air and in aluminium to reach the second child is approximately 18.65. This means that the sound wave in aluminium reaches the second child about 18.65 times faster than the sound wave in air.


In [178]:
from typing import List, Dict
from llama_index.core import VectorStoreIndex, Document
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.query_engine import BaseQueryEngine
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_parse import LlamaParse

In [185]:

class QueryProcessor:
    def __init__(self, query_engine: BaseQueryEngine):
        self.query_engine = query_engine

    def process_query(self, query: str) -> Dict[str, str]:
        response = self.query_engine.query(query)
        return {
            "response": response.response,
            "source_nodes": [node.node.get_content() for node in response.source_nodes],
        }

class InformationAnalyzer:
    def __init__(self, groq_llm):
        self.llm = groq_llm

    def analyze_response(self, initial_response: str, query: str) -> str:
        messages = [
            ChatMessage(role="system", content="You are an AI assistant tasked with analyzing responses and determining if additional information is needed."),
            ChatMessage(role="user", content=f"""
                Given the following query and initial response, determine if any additional information is required to fully answer the query. If so, specify what additional information is needed.

                Query: {query}
                Initial Response: {initial_response}

                Is additional information required? If yes, what specific information is needed?
            """)
        ]
        response = self.llm.chat(messages)
        return response.message.content

    def formulate_final_query(self, initial_response: str, additional_info: str, query: str) -> str:
        messages = [
            ChatMessage(role="system", content="You are an AI assistant tasked with formulating queries based on given information."),
            ChatMessage(role="user", content=f"""
                Given the following:
                1. Original query: {query}
                2. Initial response: {initial_response}
                3. Additional information: {additional_info}

                Formulate a new query that will help generate a complete and accurate final answer.
            """)
        ]
        response = self.llm.chat(messages)
        return response.message.content

def create_rag_agent(index: VectorStoreIndex, llm: Groq) -> tuple:
    query_engine = index.as_query_engine(
        similarity_top_k=5,
        node_postprocessors=[
            FlagEmbeddingReranker(
                top_n=3,
                model="BAAI/bge-reranker-large",
            )
        ],
        verbose=True
    )

    return query_engine, llm

def process_advanced_query(query_engine: BaseQueryEngine, llm: Groq, query: str) -> str:
    processor = QueryProcessor(query_engine)
    analyzer = InformationAnalyzer(llm)

    # Step 1: Generate initial response
    initial_result = processor.process_query(query)
    initial_response = initial_result['response']

    # Step 2: Check if additional information is required
    analysis_result = analyzer.analyze_response(initial_response, query)

    if "No additional information is required" in analysis_result:
        return initial_response

    # Step 3: Fetch additional required information
    additional_info_query = f"Find information about: {analysis_result}"
    additional_info_result = processor.process_query(additional_info_query)
    additional_info = additional_info_result['response']

    # Step 4: Formulate new query and generate final answer
    final_query = analyzer.formulate_final_query(initial_response, additional_info, query)
    final_result = processor.process_query(final_query)

    return final_result['response']

In [187]:
query_engine, llm = create_rag_agent(index_markdown_element, llm)

# Example usage
ques = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."
result = process_advanced_query(query_engine, llm, ques)
print(f"Query Result: {result}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  9.03it/s]
2024-09-29 07:03:38,586 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-29 07:03:39,166 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.76it/s]


Retrieval entering 8ea8291c-cc0c-4de4-a8a9-be49d3a6b694: TextNode
Retrieving from object TextNode with query Find information about: Yes, additional information is required to fully answer the query.

The specific information needed is the speed of sound in aluminium (v_al). Without knowing the exact value of v_al, the ratio of times taken by the sound wave in air and in aluminium to reach the second child cannot be determined.


2024-09-29 07:04:05,951 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-29 07:04:06,615 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:00<00:00,  3.74it/s]
2024-09-29 07:04:35,871 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Query Result: Let's break down the problem step by step. We need to find the ratio of times taken by the sound wave in air and in aluminium to reach the second child.

Let's assume the distance between the two children is 'd'. 

The time taken by the sound wave in air to reach the second child is t_air = d / v_air, where v_air is the speed of sound in air, which is 344 m/s.

The time taken by the sound wave in aluminium to reach the second child is t_aluminium = d / v_aluminium, where v_aluminium is the speed of sound in aluminium, which is 6420 m/s.

Now, we need to find the ratio of these two times:

t_air / t_aluminium = (d / v_air) / (d / v_aluminium) = v_aluminium / v_air = 6420 m/s / 344 m/s = 18.65

So, the ratio of times taken by the sound wave in air and in aluminium to reach the second child is approximately 18.65.


In [125]:
"""from typing import List
from pydantic import Field
from langchain.schema import BaseRetriever, Document
from llama_index.core import VectorStoreIndex
from langgraph.graph import StateGraph, END
from langchain_core.language_models import BaseLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from typing import Dict, Any
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field
from typing import Optional
from langgraph.graph import StateGraph, END
from langchain_core.language_models import BaseLLM
from langchain_core.messages import BaseMessage

from typing import Any, List, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models import BaseLLM
from langchain_core.outputs import Generation, LLMResult

from pydantic import BaseModel, Field
from typing import Any, List, Optional
from llama_index.llms.groq import Groq
from langchain.schema import HumanMessage

#from llama_index.llm_predictor.base import BaseLLM

#from llama_index.output_parsers.llm import LLMResult, Generation
#from llama_index.callbacks.base import CallbackManagerForLLMRun

# Wrapper class to adapt LlamaIndex VectorStoreIndex to LangChain BaseRetriever interface
class LlamaIndexVectorStoreRetriever(BaseRetriever):
    vector_store_index: VectorStoreIndex = Field(description="The LlamaIndex VectorStoreIndex")
    similarity_top_k: int = Field(default=5, description="Number of top similar documents to retrieve")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str) -> List[Document]:
        retriever = self.vector_store_index.as_retriever(similarity_top_k=self.similarity_top_k)
        nodes = retriever.retrieve(query)
        return [Document(page_content=node.node.get_content(), metadata=node.node.metadata) for node in nodes]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement async version if needed
        return self._get_relevant_documents(query)"""

"""class State:
    #The state of the workflow.
    def __init__(
        self,
        query: str,
        initial_response: str,
        additional_info: str = "",
        final_answer: str = "",
        needs_info: bool = False
    ):
        self.query = query
        self.initial_response = initial_response
        self.additional_info = additional_info
        self.final_answer = final_answer
        self.needs_info = needs_info

class WorkflowState(BaseModel):
    query: str
    initial_response: str
    additional_info: str = ""
    final_answer: str = ""
    needs_info: bool = False


class GroqLLMWrapper(BaseLLM, BaseModel):
    groq_llm: Groq = Field(..., description="The Groq LLM instance")

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        generations = []
        for prompt in prompts:
            response = self.groq_llm.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model="llama3-70b-8192",  # Already passed in Groq instance
                stop=stop,
                **kwargs
            )
            generations.append([Generation(text=response.choices[0].message.content)])
        return LLMResult(generations=generations)

    @property
    def _llm_type(self) -> str:
        return "groq"

    def predict_messages(self, messages: List[str], **kwargs) -> str:
    # Convert HumanMessage objects to their text content
        prompt = " ".join([msg.content if isinstance(msg, HumanMessage) else msg for msg in messages])
        response = self._generate([prompt], **kwargs)
        return response.generations[0][0].text"""


#def check_completeness(state: WorkflowState, llm: GroqLLMWrapper) -> Dict[str, WorkflowState]:
#    prompt = ChatPromptTemplate.from_template(
#        """Given the following query and initial response, determine if the response contains 
"""        all necessary information to fully answer the query. If not, specify what additional 
        information is needed.

        Query: {query}
        Initial Response: {initial_response}

        Output your response in the following format:
        Complete: [Yes/No]
        Additional Info Needed: [If No, specify what additional information is needed]
        """
#    )
    
#    parser = StructuredOutputParser.from_response_schemas([
#        ResponseSchema(name="Complete", description="Whether the response is complete (Yes/No)"),
#        ResponseSchema(name="Additional Info Needed", description="What additional information is needed, if any")
#    ])
    
#    response = llm.predict_messages(prompt.format_messages(
#        query=state.query, 
#        initial_response=state.initial_response
#    ))
    
#    parsed_response = parser.parse(response)
#    state.needs_info = parsed_response["Complete"].lower() == "no"
#    if state.needs_info:
#        state.additional_info = parsed_response["Additional Info Needed"]
    
#    return {"state": state}

"""def retrieve_additional_info(state: WorkflowState, retriever: LlamaIndexVectorStoreRetriever) -> Dict[str, WorkflowState]:
    #Retrieve additional information if needed.
    if state.needs_info:
        additional_info_query = f"Provide information about: {state.additional_info}"
        docs = retriever.get_relevant_documents(additional_info_query)
        state.additional_info = "\n".join([doc.page_content for doc in docs])
    return {"state": state}"""

#def generate_final_answer(state: WorkflowState, llm: GroqLLMWrapper) -> Dict[str, WorkflowState]:
#    prompt = ChatPromptTemplate.from_template(
#        """Given the following query, initial response, and additional information (if any),
#        generate a complete and accurate answer.0

#        Query: {query}
#        Initial Response: {initial_response}
 #       Additional Information: {additional_info}

#        Complete Answer:
#        """
#    )
    
#    state.final_answer = llm.predict_messages(prompt.format_messages(
#        query=state.query,
#        initial_response=state.initial_response,
#        additional_info=state.additional_info
#    ))
    
#    return {"state": state}

"""def create_workflow(llm: BaseLLM, retriever: LlamaIndexVectorStoreRetriever) -> StateGraph:
    workflow = StateGraph(WorkflowState)
    
    # Add nodes
    workflow.add_node("check_completeness", lambda state: check_completeness(state, llm))
    workflow.add_node("retrieve_additional_info", lambda state: retrieve_additional_info(state, retriever))
    workflow.add_node("generate_final_answer", lambda state: generate_final_answer(state, llm))
    
    # Define edges
    workflow.add_edge("check_completeness", "retrieve_additional_info")
    workflow.add_edge("retrieve_additional_info", "generate_final_answer")
    workflow.add_edge("generate_final_answer", END)
    
    # Set entry point
    workflow.set_entry_point("check_completeness")
    
    return workflow

def solve_query_with_workflow(query: str, initial_response: str, groq_llm, vector_store_index: VectorStoreIndex) -> str:
    # Pass groq_llm as a keyword argument
    llm = GroqLLMWrapper(groq_llm=groq_llm)  
    retriever_wrapper = LlamaIndexVectorStoreRetriever(vector_store_index=vector_store_index, similarity_top_k=5)
    workflow = create_workflow(llm, retriever_wrapper)
    
    # Create the runnable graph
    app = workflow.compile()
    
    # Prepare the initial state
    initial_state = WorkflowState(
        query=query,
        initial_response=initial_response
    )
    
    # Run the workflow
    final_state = app.invoke(initial_state)
    
    return final_state.final_answer"""


In [26]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [27]:
from llama_index.vector_stores.chroma import ChromaVectorStore


In [20]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")


In [24]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [25]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)


In [26]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)


In [31]:
"""def index_setup(nodes, embed_model, db_path="./chroma_db"):
    # Create or connect to the persistent ChromaDB
    db = chromadb.PersistentClient(path=db_path)
    chroma_collection = db.get_or_create_collection("quickstart")

    # Create ChromaVectorStore
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # Create storage context and index
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex(
        nodes, storage_context=storage_context, embed_model=embed_model
    )

    return index"""

In [ ]:
"""def load_index(embed_model, db_path="./chroma_db"):
    # Load the index from persistent storage
    db = chromadb.PersistentClient(path=db_path)
    chroma_collection = db.get_or_create_collection("quickstart")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

    # Create the index from the vector store
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model
    )

    return index"""

In [32]:
#nodes_index = index_setup(nodes,embed_model, db_path="./chroma_db_nodes")


In [ ]:
"""query_engine = nodes_index.as_query_engine(llm=llm
    ,similarity_top_k=3, verbose=True
)

question1 = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."

response = query_engine.query(question1)
print(response.source_nodes)"""

In [ ]:
# load from disk , nodes 
"""db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)"""

In [40]:
from llama_index.core import Settings


In [29]:
recursive_query_engine = index.as_query_engine(llm=llm
    ,similarity_top_k=5, verbose=True
)

In [ ]:
query = "Why is sound wave called a longitudinal wave?"
response = recursive_query_engine.query(query)
print(response)

In [ ]:
print(documents[-2].text[::])

In [ ]:
print(documents[-2].text[::])

In [45]:
questions = {
1 : "What is sound and how is it produced?",

2: "Describe with the help of a diagram, how compressions and rarefactions are produced in air near a source of sound.",

3: "Why is sound wave called a longitudinal wave?",

4: "Which characteristic of the sound helps you to identify your friend by his voice while sitting with others in a dark room?",

5: "Flash and thunder are produced simultaneously. But thunder is heard a few seconds after the flash is seen, why?",

6: "A person has a hearing range from 20 Hz to 20 kHz. What are the typical wavelengths of sound waves in air corresponding to these two frequencies? Take the speed of sound in air as 344 m s⁻¹.",

7: "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child.",

8: "The frequency of a source of sound is 100 Hz. How many times does it vibrate in a minute?",

9: "Does sound follow the same laws of reflection as light does? Explain.",

10: "When a sound is reflected from a distant object, an echo is produced. Let the distance between the reflecting surface and the source of sound production remains the same. Do you hear echo sound on a hotter day?",

11: "Give two practical applications of reflection of sound waves.",

12: "A stone is dropped from the top of a tower 500 m high into a pond of water at the base of the tower. When is the splash heard at the top? Given, g = 10 m s⁻² and speed of sound = 340 m s⁻¹.",

13: "A sound wave travels at a speed of 339 m s⁻¹. If its wavelength is 1.5 cm, what is the frequency of the wave? Will it be audible?",

14: "What is reverberation? How can it be reduced?",

15: "What is loudness of sound? What factors does it depend on?",

16: "How is ultrasound used for cleaning?",

17: "Explain how defects in a metal block can be detected using ultrasound."
}


In [38]:
answers = []

for value in questions.values(): 
    repo = recursive_query_engine.query(value)
    answers.append(repo)

In [ ]:
print(answers[6])

In [ ]:
questions.items()

In [51]:
ans = {}
for key, value in questions.items():
    ans[key] = recursive_query_engine.query(value)

In [ ]:
print(ans[14])